In [ ]:
from plotnine import ggplot, aes, geom_point, labs, facet_wrap, geom_vline, theme_minimal, theme, element_text, geom_errorbarh
import pandas as pd

In [ ]:
df_detailed   = pd.read_csv('../acc_rate_csvs/detailed.csv')
df_detailed['dataset'] = 'detailed'
df_systematic = pd.read_csv('../acc_rate_csvs/systematic.csv')
df_systematic['dataset'] = 'systematic'
df = pd.concat([df_detailed, df_systematic])

avg_df = df.groupby(['dataset', 'method', 'feature engineer']).agg(avg_acc=('acc', 'mean'), std=('acc', 'std')).reset_index()

In [ ]:
acc_compare_df = avg_df[avg_df['feature engineer']=='yes'].copy()
acc_compare_df['lower_bound'] = acc_compare_df['avg_acc'] - acc_compare_df['std']
acc_compare_df['upper_bound'] = acc_compare_df['avg_acc'] + acc_compare_df['std']
acc_compare_df['method'] = pd.Categorical(acc_compare_df['method'], categories=['BIC.1', 'linear.1', 'linear.2', 'linear.4', 'linear.117', 'mlp.1', 'mlp.2', 'mlp.4'], ordered=True)

In [ ]:
max_acc_per_dataset = acc_compare_df.groupby('dataset')['upper_bound'].max().reset_index()
max_acc_per_dataset.loc[:, 'upper_bound'] += 0.2

plot_combined = (ggplot(acc_compare_df, aes(x='avg_acc', y='method')) +
                 geom_point(shape="o", size=2, fill="white") +
                 geom_errorbarh(aes(xmin='lower_bound', xmax='upper_bound'), height=0.1) +
                 labs(x="accuracy percentage",
                      y="method") +
                 facet_wrap('~dataset', ncol=acc_compare_df['dataset'].nunique(), scales='free_x') +
                 theme_minimal() +
                 theme(legend_position='bottom', text=element_text(size=8)) +
                 theme(aspect_ratio=0.9) +
                 geom_vline(data=max_acc_per_dataset, mapping=aes(xintercept='upper_bound'), color='black') + 
                 theme(figure_size=(6, 3))
                 )

print(plot_combined)

In [ ]:
plot_combined.save("acc_compare.pdf")
plot_combined.save("acc_compare.jpg")